In [1]:
import cv2 as cv
import numpy as np
import csv
import os
from os import walk
import random
from datetime import datetime

In [2]:
folder_path = "../../03 Data/DOTA_tennis-court/"
training_imgs = "DOTA_train.txt"
validation_imgs = "DOTA_val.txt"

train_file = open(folder_path+training_imgs)
train_files = list(train_file)
train_file.close()
for i, img in enumerate(train_files):
    train_files[i] = img.strip('.png\n')+'.txt'

val_file = open(folder_path + validation_imgs)
val_files = list(val_file)
val_file.close()
for i, img in enumerate(val_files):
    val_files[i] = img.strip('.png\n')+'.txt'



In [3]:
now = datetime.now()
date = now.strftime("%d-%m")

names = ['train_DOTA'+date+'.csv', 'val_DOTA'+date+'.csv']
combined_files = [train_files, val_files]
wh_list = []
# create csv of train-set and val-set for feeding to the network
for i in range(len(names)):
    with open(names[i], 'w', newline='') as open_file:
        writer = csv.writer(open_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        with open('list_of_img_in_'+names[i], 'w') as f:
            for file in combined_files[i]:
                #Create annotation list
                train_file = open(folder_path+"DOTA_Annotations/"+file)
                train_files = list(train_file)
                train_file.close()
                annotations = [anno.split() for anno in train_files[2:]]
                image_name= file.strip('.txt')+'.png'

                f.write("%s\n" % image_name)
                    # check if image contains annotations.
                if len(annotations) != 0:
                    for annotation in annotations:
                        if(annotation[8] == 'tennis-court'):
                            rot_rect = cv.minAreaRect(np.float32(annotations[1][:8]).reshape(4,2))
                            bp = cv.boxPoints(rot_rect) # bp = box points
                            #p1 = [rot_rect[0]+]
                            # make sure that width and height are consistent
                            if rot_rect[-2][0] < rot_rect[-2][1]:
                                rot_rect = (rot_rect[0],(rot_rect[-2][1],rot_rect[-2][0]),rot_rect[-1]+90)
                            # make sure that no angle is above 90 or below -90
                            wh_list.append(list(rot_rect[1]))
                            if rot_rect[-1] > 90:
                                rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]-180)
                            if rot_rect[-1] < -90:
                                rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]+180)
                            class_name = annotation[8]
                            writer.writerow([folder_path + image_name, bp[0][0], bp[0][1], bp[1][0], bp[1][1],
                                             bp[2][0], bp[2][1], bp[3][0], bp[3][1], class_name,rot_rect[-1]])
                            # format: img_path, p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y, class_name, angle
                else:
                    writer.writerow([path + image_name, ",,,,,,,,,,"])

In [31]:
train_file = open('P0000.txt')
train_files = list(train_file)
train_file.close()
annotations = [anno.split() for anno in train_files[2:]]
print(annotations[1][:8])
print(cv.minAreaRect(np.float32(annotations[1][:8]).reshape(4,2)))

['1464.0', '2131.0', '1472.0', '2133.0', '1470.0', '2145.0', '1462.0', '2143.0']
((1467.0, 2138.0), (12.823122024536133, 8.219949722290039), -80.53767395019531)
